In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
df=pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx')

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
col_names={'X1':'Relative_Compactness', 
           'X2': 'Surface_Area',
           'X3': 'Wall_Area', 'X4': 'Roof_Area', 'X5': 'Overall_Height',
           'X6': 'Orientation', 'X7': 'Glazing_Area',
           'X8': 'Glazing_Area_Distribution',
           'Y1': 'Heating_Load', 'Y2': 'Cooling_Load'}
df.rename(columns= col_names, inplace=True)
df

In [ ]:
simple_linear_reg_df = df[['Relative_Compactness', 'Cooling_Load']].sample(15, random_state=2)

In [ ]:
simple_linear_reg_df2 = df[['Overall_Height',"Heating_Load"]].sample(15, random_state=2)

In [ ]:
sns.regplot(x='Relative_Compactness', y= "Cooling_Load", data= simple_linear_reg_df);

In [ ]:
sns.regplot(x='Overall_Height', y= "Heating_Load", data= simple_linear_reg_df2);

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(df.corr(),annot= True)

In [ ]:
scaler= MinMaxScaler()

In [ ]:
normalised_df= pd.DataFrame(scaler.fit_transform(df), columns= df.columns)

In [ ]:
features_df= normalised_df.drop(columns= ['Heating_Load', 'Cooling_Load'])

In [ ]:
heating_target= normalised_df['Heating_Load']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features_df, heating_target,
test_size=0.3, random_state=1)

In [ ]:
linear_model= LinearRegression()

In [ ]:
linear_model.fit(x_train, y_train)

In [ ]:
pred_value = linear_model.predict(x_test)

In [ ]:
mae = mean_absolute_error(y_test, pred_value)
mae=round(mae, 3) 

In [ ]:
mae

In [ ]:
mse = mean_squared_error(y_test, pred_value)
mse=round(mse, 3) 
mse

In [ ]:
# residual sum of square
rss= np.sum(np.square(y_test-pred_value))
round(rss,3)

In [ ]:
### rmse=np.sqrt(mse)
round(rmse,3)

In [42]:
r2= r2_score(y_test, pred_value)
round(r2, 3)

0.894

## Ridge Regression

In [44]:
ridge_reg= Ridge(alpha=0.5)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.5)

## Lasso and ElasticNet

In [47]:
lasso_reg= Lasso(alpha=0.001)

In [49]:
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001)

In [54]:
regr= ElasticNet(alpha=0.001)
regr.fit(x_train, y_train)

ElasticNet(alpha=0.001)

In [62]:
def get_weights(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [63]:
linear_model_weights= get_weights(linear_model, x_train, "Linear_Model_Weight" )
ridge_weights_df= get_weights(ridge_reg, x_train, "Ridge Weight")
lasso_weights_df= get_weights(lasso_reg, x_train, "Lasso Weight")
elastic_weights_df= get_weights(regr, x_train, "Elastic Weight")

In [64]:
final_weights= pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights= pd.merge(final_weights, lasso_weights_df, on= "Features")
final_weights= pd.merge(final_weights, elastic_weights_df, on='Features')

In [65]:
final_weights

,Features,Linear_Model_Weight,Ridge Weight,Lasso Weight,Elastic Weight
0,Surface_Area,-2.388039e+12,-0.062275,0.000000,0.000000
1,Relative_Compactness,-6.064125e-01,-0.283471,-0.027719,-0.069218
2,Orientation,2.440705e-03,0.003369,0.000000,0.000000
3,Glazing_Area_Distribution,2.678929e-02,0.029088,0.021431,0.024931
4,Glazing_Area,2.168792e-01,0.212449,0.206132,0.208971
5,Overall_Height,3.924579e-01,0.442467,0.463482,0.487470
6,Wall_Area,1.393023e+12,0.103061,0.200087,0.183070
7,Roof_Area,1.791029e+12,-0.163192,-0.000000,-0.000000
